In [1]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib widget
plt.close('all')
import numpy as np
from sklearn.preprocessing import StandardScaler
from random import choice
import imageio
import os
import cv2
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit

path_save = r"C:\Users\coren\AMOLF-SHIMIZU Dropbox\Corentin Bisot\Applications\princesync\data_tables\time_plate_infos.pick"
time_plate_info = pd.read_pickle(path_save)
plt.style.use('presentation.mplstyle')

In [2]:
import matplotlib as mpl
cmap1 = mpl.cm.get_cmap('spring')
cmap2 = mpl.cm.get_cmap('winter')
def S(t,lamb,C,t0):
    return(C*(1/(1+np.exp(lamb*(t0-t)))))

In [6]:
list(table.columns)

['num_trunks',
 'area',
 'area_sep_comp',
 'num_tips',
 'num_nodes',
 'area_study',
 'num_tips_study',
 'num_nodes_study',
 'tot_length_study',
 'out_study',
 'date',
 'Plate',
 'path_exp',
 'folder_analysis',
 'ring_density_incr-100_index-0',
 'ring_density_incr-100_index-1',
 'ring_density_incr-100_index-2',
 'ring_density_incr-100_index-3',
 'ring_density_incr-100_index-4',
 'ring_density_incr-100_index-5',
 'ring_density_incr-100_index-6',
 'ring_density_incr-100_index-7',
 'ring_density_incr-100_index-8',
 'ring_density_incr-100_index-9',
 'ring_density_incr-100_index-10',
 'ring_density_incr-100_index-11',
 'ring_density_incr-100_index-12',
 'ring_density_incr-100_index-13',
 'ring_density_incr-100_index-14',
 'ring_density_incr-100_index-15',
 'ring_density_incr-100_index-16',
 'ring_density_incr-100_index-17',
 'ring_density_incr-100_index-18',
 'ring_density_incr-100_index-19',
 'ring_bas_density_incr-100_index-0',
 'ring_bas_density_incr-100_index-1',
 'ring_bas_density_incr-

In [26]:
plt.close('all')
plate = 76
fig, ax = plt.subplots()

ax2 = ax.twinx()
maxi = 12
table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
table= table.set_index('t')

ts = []
ys = []
ys2 = []
max_area = 50
Cs = []
lambs = [] 
indexes = []
t0s = []
ds = []
for index in range(4,maxi):
    column = f"ring_density_incr-100_index-{index}"
    column2 = f"ring_active_tips_density_incr-100_index-{index}"
    
    start = np.min(table.loc[table[column]>=1000]['time_since_begin'])
    if not np.isnan(start):
        table[f'time_since_begin_{index}']=table['time_since_begin']-start
        area = np.sqrt(table["area_sep_comp"][0]+100*index)

        ax.scatter(table[f'time_since_begin_{index}'],table[column],alpha=0.5,color=cmap2(area/max_area))
        ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.5,color=cmap1(area/max_area))
        selection_fit = table
        popt0, pcov = curve_fit(S, selection_fit[f'time_since_begin_{index}'],selection_fit[column] ,bounds = ([0,0,-np.inf],3*[np.inf]),p0=[1,1,0])
        lamb,C,t0 = list(popt0)
        Cs.append(C)
        lambs.append(lamb)
        indexes.append(index)
        t0s.append(t0+start)
        ds.append(int(area/np.sqrt((np.pi/2))))
        ax.plot(table[f'time_since_begin_{index}'],S(table[f'time_since_begin_{index}'],lamb,C,t0),color=cmap2(area/max_area),label=f'd = {int(area/np.sqrt((np.pi/2)))}mm')
        ts +=table[f'time_since_begin_{index}'].to_list()
        ys += table[column].to_list()
        ys2+=table[column2].astype(float).to_list()
df = pd.DataFrame((np.array((ts,ys,ys2))).transpose(),columns=('ts','ys','ys2'))
factor = 4
df['ts_round'] = (df['ts']/factor).astype(int)*factor
meancurve = df.groupby('ts_round')['ys'].mean()
# ax.plot(meancurve.index,meancurve,color="blue",linewidth = 5)
meancurve2 = df.groupby('ts_round')['ys2'].mean()
# ax2.plot(meancurve.index,meancurve2,color="red",linewidth = 5)
# ax2.set_color('red')
ax.set_xlim((-30,30))
ax.set_ylabel('network density ($\mu m.mm^{-2}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel('active tips density ($mm^{-2}$)')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
ax.legend(fontsize = 8)
plt.tight_layout()
plt.savefig("Figure/plate95fixedpoint.svg")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
fig, ax = plt.subplots()
ax.scatter(t0s,ds)
ax.set_xlabel('time (h)')
ax.set_ylabel('d (mm)')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
fig, ax = plt.subplots()
vs = np.gradient(ds,t0s)
ax.scatter(t0s,vs)
ax.set_xlabel('time (h)')
ax.set_ylabel('c (mm.h-1)')
ax.set_ylim((0,1))
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
plt.close('all')
plate = 94
fig, ax = plt.subplots()

ax2 = ax.twinx()
maxi = 12
table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
table= table.set_index('t')

ts = []
ys = []
ys2 = []
max_area = 50
Cs = []
lambs = [] 
indexes = []
for index in range(0,maxi):
    column = f"ring_density_incr-100_index-{index}"
    column2 = f"ring_active_tips_density_incr-100_index-{index}"
    column2 = f"ring_bas_density_incr-100_index-{index}"
    
    start = np.min(table.loc[table[column]>=1000]['time_since_begin'])
    if not np.isnan(start):
        table[f'time_since_begin_{index}']=table['time_since_begin']-start
        area = np.sqrt(table["area_sep_comp"][0]+100*index)

        ax.scatter(table[f'time_since_begin_{index}'],table[column],alpha=0.5,color=cmap2(area/max_area))
        ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.5,color=cmap1(area/max_area))
#         ax2.plot(table[f'time_since_begin_{index}'],table[column2]/(table[column]+1),alpha=0.5,color=cmap1(area/max_area))
        
        selection_fit = table
        popt0, pcov = curve_fit(S, selection_fit[f'time_since_begin_{index}'],selection_fit[column] ,bounds = ([0,0,-np.inf],3*[np.inf]),p0=[1,1,0])
        lamb,C,t0 = list(popt0)
        Cs.append(C)
        lambs.append(lamb)
        indexes.append(index)
        ax.plot(table[f'time_since_begin_{index}'],S(table[f'time_since_begin_{index}'],lamb,C,t0),color=cmap2(area/max_area),label=f'd = {int(area/np.sqrt((np.pi/2)))}mm')
        ts +=table[f'time_since_begin_{index}'].to_list()
        ys += table[column].to_list()
        ys2+=table[column2].astype(float).to_list()
df = pd.DataFrame((np.array((ts,ys,ys2))).transpose(),columns=('ts','ys','ys2'))
factor = 4
df['ts_round'] = (df['ts']/factor).astype(int)*factor
meancurve = df.groupby('ts_round')['ys'].mean()
# ax.plot(meancurve.index,meancurve,color="blue",linewidth = 5)
meancurve2 = df.groupby('ts_round')['ys2'].mean()
# ax2.plot(meancurve.index,meancurve2,color="red",linewidth = 5)
# ax2.set_color('red')
ax.set_xlim((-30,30))
# ax2.set_ylim((0,0.5))

ax.set_ylabel('network density ($\mu m.mm^{-2}$)')
ax.set_xlabel('shifted time ($h$)')
# ax2.set_ylabel('bas/L ratio ($mm^{-2}$)')
ax2.set_ylabel('bas length ($mm^{-2}$)')

ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
ax.legend(fontsize = 8)
plt.tight_layout()
plt.savefig("Figure/plate95fixedpoint.svg")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
fig, ax = plt.subplots()
ax.scatter(indexes,lambs)
# ax.set_ylim(0,20)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
plt.close('all')
fig, ax = plt.subplots()

ax2 = ax.twinx()
maxi = 12
max_area = 50
Cs = []
lambs = [] 
indexes = []
plates = []
for plate in [94,152,792,799,800,91,80]:
    table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
    table= table.set_index('t')
    
    ts = []
    ys = []
    ys2 = []
    for index in range(1,maxi):
        column = f"ring_density_incr-100_index-{index}"
        column2 = f"ring_active_tips_density_incr-100_index-{index}"
        start = np.min(table.loc[table[column]>=1000]['time_since_begin'])
        if not np.isnan(start):
            table[f'time_since_begin_{index}']=table['time_since_begin']-start
            area = np.sqrt(table["area_sep_comp"][0]+100*index)
            selection_fit = table
            popt0, pcov = curve_fit(S, selection_fit[f'time_since_begin_{index}'],selection_fit[column] ,bounds = ([0,0,-np.inf],3*[np.inf]),p0=[1,1,0])
            lamb,C,t0 = list(popt0)
            Cs.append(C)
            lambs.append(lamb)
            indexes.append(index)
            plates.append(plate)
            ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.5,color=cmap2(area/max_area))
            ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.5,color=cmap1(area/max_area))

            ts +=table[f'time_since_begin_{index}'].to_list()
            ys += table[column].to_list()
            ys2+=table[column2].astype(float).to_list()
df = pd.DataFrame((np.array((ts,ys,ys2))).transpose(),columns=('ts','ys','ys2'))
factor = 4
df['ts_round'] = (df['ts']/factor).astype(int)*factor
meancurve = df.groupby('ts_round')['ys'].mean()
# ax.plot(meancurve.index,meancurve,color="blue",linewidth = 5)
meancurve2 = df.groupby('ts_round')['ys2'].mean()
# ax2.plot(meancurve.index,meancurve2,color="red",linewidth = 5)
# ax2.set_color('red')
ax.set_xlim((-30,30))
ax.set_ylabel('network density ($\mu m.mm^{-2}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel('active tips density ($mm^{-2}$)')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\pandas\core\arraylike.p

In [15]:
cmap3 = mpl.cm.get_cmap('jet')

In [24]:
cmap3(128)

(0.4901960784313725, 1.0, 0.4775458570524984, 1.0)

In [56]:
colors = {plate : cmap3(np.random.random()) for plate in plates}

In [53]:
colors

{94: (0.21189120809614148, 1.0, 0.7558507273877295, 1.0),
 152: (0.08538899430740036, 1.0, 0.8823529411764706, 1.0),
 792: (0.0, 0.0, 0.803030303030303, 1.0),
 799: (0.0, 0.0, 0.517825311942959, 1.0),
 800: (0.43959519291587595, 1.0, 0.5281467425679949, 1.0),
 91: (0.3383934218848829, 1.0, 0.629348513598988, 1.0)}

In [65]:
fig, ax = plt.subplots()
data = pd.DataFrame(np.array((indexes,lambs,plates)).transpose(),columns=('index','lamb',"plate"))
data = data.loc[data['lamb']<1]
data.groupby('index')['lamb'].mean()
group = data.groupby('index')['lamb']
ax.plot(group.mean()) 

ax.scatter(data['index'],data['lamb'],alpha = 0.5,color = [colors[plate] for plate in data['plate']])
ax.errorbar(group.mean().index,group.mean(),group.std()/np.sqrt(group.count()))
ax.set_xlabel("ring index")
ax.set_ylabel("$\lambda (h^{-1})$")
plt.savefig("lambdas.svg")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [64]:
fig, ax = plt.subplots()
data = pd.DataFrame(np.array((indexes,lambs,Cs,plates)).transpose(),columns=('index','lamb','C',"plate"))
y = 'C'
data = data.loc[data[y]<5000]
data.groupby('index')[y].mean()
group = data.groupby('index')[y]
ax.plot(group.mean()) 

ax.scatter(data["index"],data[y],alpha = 0.5,color = [colors[plate] for plate in data['plate']])
ax.errorbar(group.mean().index,group.mean(),group.std()/np.sqrt(group.count()))
ax.set_xlabel("ring index")
ax.set_ylabel("$C (\mu m.mm{-2})$")
plt.savefig("Cs.svg")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [46]:
data

,index,lamb,C,plate


In [14]:
data

,index,lamb,plate
0,1.0,0.175676,94.0
1,2.0,0.218683,94.0
2,3.0,0.302389,94.0
3,4.0,0.250898,94.0
4,5.0,0.215871,94.0
5,6.0,0.204775,94.0
6,7.0,0.177028,94.0
7,8.0,0.215575,94.0
8,9.0,0.195623,94.0
9,10.0,0.221512,94.0


In [8]:
data = pd.DataFrame(np.array((indexes,lambs)).transpose(),columns=('index','lamb'))
data = data.loc[data['lamb']<1]
data.groupby('index')['lamb'].mean()

index
1.0     0.123757
2.0     0.155933
3.0     0.193231
4.0     0.194209
5.0     0.211113
6.0     0.207487
7.0     0.142126
8.0     0.141445
9.0     0.154511
10.0    0.153066
11.0    0.129204
Name: lamb, dtype: float64

In [10]:
list(table.columns)

['t',
 'tot_length',
 'date',
 'Plate',
 'path_exp',
 'folder_analysis',
 'num_trunks',
 'area',
 'area_sep_comp',
 'num_tips',
 'num_nodes',
 'area_study',
 'num_tips_study',
 'num_nodes_study',
 'tot_length_study',
 'out_study',
 'ring_density_incr-100_index-0',
 'ring_density_incr-100_index-1',
 'ring_density_incr-100_index-2',
 'ring_density_incr-100_index-3',
 'ring_density_incr-100_index-4',
 'ring_density_incr-100_index-5',
 'ring_density_incr-100_index-6',
 'ring_density_incr-100_index-7',
 'ring_density_incr-100_index-8',
 'ring_density_incr-100_index-9',
 'ring_density_incr-100_index-10',
 'ring_density_incr-100_index-11',
 'ring_density_incr-100_index-12',
 'ring_density_incr-100_index-13',
 'ring_density_incr-100_index-14',
 'ring_density_incr-100_index-15',
 'ring_density_incr-100_index-16',
 'ring_density_incr-100_index-17',
 'ring_density_incr-100_index-18',
 'ring_density_incr-100_index-19',
 'ring_anas_density_incr-100_index-0',
 'ring_anas_density_incr-100_index-1',
 

In [33]:
plt.close('all')
plate = 94
table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
fig, ax = plt.subplots()
ts = []
ys = []
ys2 = []
ax2 = ax.twinx()

for index in range(20):
    column = f"ring_density_incr-100_index-{index}"
    column2 = f"ring_bas_density_incr-100_index-{index}"
    start = np.min(table.loc[table[column]>=1000]['time_since_begin'])
    if not np.isnan(start):
        table[f'time_since_begin_{index}']=table['time_since_begin']-start
        ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
        ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')

        ts +=table[f'time_since_begin_{index}'].to_list()
        ys += table[column].to_list()
        ys2+=table[column2].astype(float).to_list()
df = pd.DataFrame((np.array((ts,ys,ys2))).transpose(),columns=('ts','ys','ys2'))
factor = 4
df['ts_round'] = (df['ts']/factor).astype(int)*factor
meancurve = df.groupby('ts_round')['ys'].mean()
ax.plot(meancurve.index,meancurve,color="blue",linewidth = 5)
meancurve2 = df.groupby('ts_round')['ys2'].mean()
ax2.plot(meancurve.index,meancurve2,color="red",linewidth = 5)
# ax2.set_color('red')
ax.set_xlim((-30,30))
ax2.set_ylim((-25,650))
ax.set_ylabel('network density ($\mu m.mm^{-2}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel('bas density ($\mu m.mm^{-2}$)')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
plt.close('all')
plate = 94
table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
fig, ax = plt.subplots()
ts = []
ys = []
ys2 = []
ax2 = ax.twinx()

for index in range(20):
    column = f"ring_branch_density_incr-100_index-{index}"
    column2 = f"ring_active_tips_density_incr-100_index-{index}"
    start = np.min(table.loc[table[f"ring_density_incr-100_index-{index}"]>=1000]['time_since_begin'])
    if not np.isnan(start):
        table[f'time_since_begin_{index}']=table['time_since_begin']-start
        ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
        ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')

        ts +=table[f'time_since_begin_{index}'].to_list()
        ys += table[column].astype(float).to_list()
        ys2+=table[column2].astype(float).to_list()
df = pd.DataFrame((np.array((ts,ys,ys2))).transpose(),columns=('ts','ys','ys2'))
factor = 4
df['ts_round'] = (df['ts']/factor).astype(int)*factor
meancurve = df.groupby('ts_round')['ys'].mean()
ax.plot(meancurve.index,meancurve,color="blue",linewidth = 5)
meancurve2 = df.groupby('ts_round')['ys2'].mean()
ax2.plot(meancurve.index,meancurve2,color="red",linewidth = 5)
# ax2.set_color('red')
ax.set_xlim((-30,30))
ax.set_ylim((-0.00030,0.010))
ax.set_ylabel('new branch density ($mm^{-2}.h^{-1}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel('active tips density ($mm^{-2}$)')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
plt.close('all')
plate = 94
table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
fig, ax = plt.subplots()
ts = []
ys = []
ys2 = []
ax2 = ax.twinx()

for index in range(20):
    column = f"ring_anas_density_incr-100_index-{index}"
    column2 = f"ring_active_tips_density_incr-100_index-{index}"
    start = np.min(table.loc[table[f"ring_density_incr-100_index-{index}"]>=1000]['time_since_begin'])
    if not np.isnan(start):
        table[f'time_since_begin_{index}']=table['time_since_begin']-start
        ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
        ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')

        ts +=table[f'time_since_begin_{index}'].to_list()
        ys += table[column].astype(float).to_list()
        ys2+=table[column2].astype(float).to_list()
df = pd.DataFrame((np.array((ts,ys,ys2))).transpose(),columns=('ts','ys','ys2'))
factor = 4
df['ts_round'] = (df['ts']/factor).astype(int)*factor
meancurve = df.groupby('ts_round')['ys'].mean()
ax.plot(meancurve.index,meancurve,color="blue",linewidth = 5)
meancurve2 = df.groupby('ts_round')['ys2'].mean()
ax2.plot(meancurve.index,meancurve2,color="red",linewidth = 5)
# ax2.set_color('red')
ax.set_xlim((-30,30))
ax.set_ylim((0,0.0020))
ax2.set_ylim((0,0.25))

ax.set_ylabel('anastomosis density ($mm^{-2}.h^{-1}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel('active tips density ($mm^{-2}$')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
plt.close('all')
plate = 94
table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
fig, ax = plt.subplots()
ts = []
ys = []
ys2 = []
ax2 = ax.twinx()

for index in range(15):
    column = f"mean_speed_incr-100_index-{index}"
    column2 = f"ring_active_tips_density_incr-100_index-{index}"
    column_speed = f"mean_speed_incr-100_index-{index}"
    
    start = np.min(table.loc[table[f"ring_density_incr-100_index-{index}"]>=1000]['time_since_begin'])
    if not np.isnan(start):
        table[f'time_since_begin_{index}']=table['time_since_begin']-start
        table = table.loc[(table[column_speed]<=400) | (table[column_speed].isnull()==True)]
        
        ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
        ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')

        ts +=table[f'time_since_begin_{index}'].to_list()
        ys += table[column].astype(float).to_list()
        ys2+=table[column2].astype(float).to_list()
df = pd.DataFrame((np.array((ts,ys,ys2))).transpose(),columns=('ts','ys','ys2'))
factor = 4
df['ts_round'] = (df['ts']/factor).astype(int)*factor
meancurve = df.groupby('ts_round')['ys'].mean()
ax.plot(meancurve.index,meancurve,color="blue",linewidth = 5)
meancurve2 = df.groupby('ts_round')['ys2'].mean()
ax2.plot(meancurve.index,meancurve2,color="red",linewidth = 5)
# ax2.set_color('red')
ax.set_xlim((-30,30))
ax.set_ylim((100,350))
ax.set_ylabel('new branch density ($mm^{-2}.h{-1}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel('active tips density ($mm^{-2}$)')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
plt.close('all')
fig, ax = plt.subplots()

ax2 = ax.twinx()
for plate in [94,152,792,799,800,91,76]:
    table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
    ts = []
    ys = []
    ys2 = []


    for index in range(20):
        column = f"ring_density_incr-100_index-{index}"
        column2 = f"ring_active_tips_density_incr-100_index-{index}"
        start = np.min(table.loc[table[column]>=1000]['time_since_begin'])
        if not np.isnan(start):
            table[f'time_since_begin_{index}']=table['time_since_begin']-start
#             ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
#             ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')

            ts +=table[f'time_since_begin_{index}'].to_list()
            ys += table[column].to_list()
            ys2+=table[column2].astype(float).to_list()
    df = pd.DataFrame((np.array((ts,ys,ys2))).transpose(),columns=('ts','ys','ys2'))
    factor = 4
    df['ts_round'] = (df['ts']/factor).astype(int)*factor
    meancurve = df.groupby('ts_round')['ys'].mean()
    ax.plot(meancurve.index,meancurve,label=plate)
    meancurve2 = df.groupby('ts_round')['ys2'].mean()
    ax2.plot(meancurve.index,meancurve2,label=plate,linestyle="dashed")
# ax2.set_color('red')
ax.set_xlim((-30,30))
ax.set_ylabel('network density ($\mu m.mm^{-2}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel('active tips density ($mm^{-2}$)')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
plt.legend()
plt.savefig("replicate.svg")
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
plt.close('all')
fig, ax = plt.subplots()

ax2 = ax.twinx()
for plate in [94,152,792,799,800,91]:
    table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
    ts = []
    ys = []
    ys2 = []


    for index in range(15):
        column3 = f"ring_density_incr-100_index-{index}"
        column2 = f"ring_active_tips_density_incr-100_index-{index}"
        start = np.min(table.loc[table[column3]>=1000]['time_since_begin'])
        column = f"mean_speed_incr-100_index-{index}"
        column_speed = f"mean_speed_incr-100_index-{index}"
        if not np.isnan(start):
            table[f'time_since_begin_{index}']=table['time_since_begin']-start
            table = table.loc[(table[column_speed]<=400) | (table[column_speed].isnull()==True)]
            
#             ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
#             ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')

            ts +=table[f'time_since_begin_{index}'].to_list()
            ys += table[column].astype(float).to_list()
            ys2+=table[column2].astype(float).to_list()
    df = pd.DataFrame((np.array((ts,ys,ys2))).transpose(),columns=('ts','ys','ys2'))
    factor = 4
    df['ts_round'] = (df['ts']/factor).astype(int)*factor
    meancurve = df.groupby('ts_round')['ys'].mean()
    ax.plot(meancurve.index,meancurve,label=plate)
    meancurve2 = df.groupby('ts_round')['ys2'].mean()
    ax2.plot(meancurve.index,meancurve2,label=plate,linestyle="dashed")
# ax2.set_color('red')
ax.set_xlim((-30,30))
ax.set_ylim((100,350))

ax.set_ylabel('mean speed ($\mu m.h^{-1}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel('active tips density ($mm^{-2}$')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
plt.close('all')
fig, ax = plt.subplots()
tstot = []
ystot = []
ys2tot = []
ys3tot = []

ax2 = ax.twinx()
for plate in [94,152,792,799,800,91]:
    table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
    ts = []
    ys = []
    ys2 = []
    for index in range(20):
        column = f"ring_density_incr-100_index-{index}"
#         column = f"ring_active_tips_density_incr-100_index-{index}"
        column_speed = f"mean_speed_incr-100_index-{index}"
        
        column2 = f"ring_active_tips_density_incr-100_index-{index}"
        column3 = f"mean_speed_incr-100_index-{index}"

        align_col = f"ring_density_incr-100_index-{index}"
        start = np.min(table.loc[table[align_col]>=1000]['time_since_begin'])
        if not np.isnan(start):
            table[f'time_since_begin_{index}']=table['time_since_begin']-start
            table = table.loc[(table[column_speed]<=400) | (table[column_speed].isnull()==True)]
            
#             ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
#             ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')
            
            x = f'time_since_begin_{index}'
            deb = np.min(table[x])
            end = np.max(table[x])
            time = np.linspace(deb,end,1000)
            tstot +=list(time)
            f = interp1d(table[x], table[column])
            ystot += list(f(time))
            f = interp1d(table[x], table[column2])
            ys2tot+=list(f(time))
            f = interp1d(table[x], table[column3])
            ys3tot+=list(f(time))
df = pd.DataFrame((np.array((tstot,ystot,ys2tot,ys3tot))).transpose(),columns=('ts','ys','ys2','ys3'))
factor = 4
df['ts_round'] = np.floor(df['ts']/factor)*factor
meancurve = df.groupby('ts_round')['ys'].mean()
grad = np.gradient(meancurve,meancurve.index)

meancurve2 = df.groupby('ts_round')['ys2'].mean()
meancurve3 = df.groupby('ts_round')['ys3'].mean()
ax2.plot(meancurve3.index,meancurve3,linewidth = 5,color='red')

ax.plot(meancurve2.index,meancurve2*meancurve3,linewidth = 5)
ax2.set_ylim((140,250))

# ax2.set_color('red')
ax.set_xlim((-30,30))
ax.set_ylabel('n ($mm^{-2}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel('v ($\mu m.h^{-1}$')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
# plt.close('all')
# fig, ax = plt.subplots()
tstot = []
ystot = []
ys2tot = []
# ax2 = ax.twinx()
for plate in [94,152,792,799,800,91]:
    table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
    ts = []
    ys = []
    ys2 = []
    for index in range(20):
        column2 = f"mean_speed_incr-100_index-{index}"
        column = f"ring_active_tips_density_incr-100_index-{index}"
        column_speed = f"mean_speed_incr-100_index-{index}"
        
        start = np.min(table.loc[table[f"ring_density_incr-100_index-{index}"]>=1000]['time_since_begin'])
        if not np.isnan(start):
            table[f'time_since_begin_{index}']=table['time_since_begin']-start
            table = table.loc[(table[column2]<=400) | (table[column2].isnull()==True)]
#             ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
#             ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')

            tstot +=table[f'time_since_begin_{index}'].to_list()
            ystot += table[column].astype(float).to_list()
            ys2tot+=table[column2].astype(float).to_list()
            ts +=table[f'time_since_begin_{index}'].to_list()
            ys += table[column].astype(float).to_list()
            ys2+=table[column2].astype(float).to_list()
    df = pd.DataFrame((np.array((ts,ys,ys2))).transpose(),columns=('ts','ys','ys2'))
    factor = 4
    df['ts_round'] = (df['ts']/factor).astype(int)*factor
    meancurve = df.groupby('ts_round')['ys'].mean()
#     ax.plot(meancurve.index,meancurve,color='blue',alpha=0.5)
    meancurve2 = df.groupby('ts_round')['ys2'].mean()
    ax2.plot(meancurve.index,meancurve2,color='red',alpha = 0.5)
# df = pd.DataFrame((np.array((tstot,ystot,ys2tot))).transpose(),columns=('ts','ys','ys2'))
# factor = 4
# df['ts_round'] = (df['ts']/factor).astype(int)*factor
# meancurve = df.groupby('ts_round')['ys'].mean()
# ax.plot(meancurve.index,meancurve,linewidth = 5)
# meancurve2 = df.groupby('ts_round')['ys2'].mean()
# ax2.plot(meancurve.index,meancurve2,linewidth = 5,color='red')
# # ax2.set_color('red')
# ax.set_xlim((-30,30))
# ax2.set_ylabel('mean speed ($\mu m.h^{-1}$)')
# ax.set_xlabel('shifted time ($h$)')
# ax.set_ylabel('active tips density ($mm^{-2}$)')
# ax.tick_params(axis='y', colors='blue')
# ax2.tick_params(axis='y', colors='red')
# ax2.set_ylim((100,350))

# plt.legend()

In [14]:
plt.close('all')
fig, ax = plt.subplots()
tstot = []
ystot = []
ys2tot = []
ys3tot = []

ax2 = ax.twinx()
for plate in [94,152,792,799,800,91]:
    table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
    ts = []
    ys = []
    ys2 = []
    for index in range(20):
        column = f"ring_density_incr-100_index-{index}"
#         column = f"ring_active_tips_density_incr-100_index-{index}"
        
        column2 = f"ring_active_tips_density_incr-100_index-{index}"
        column3 = f"mean_speed_incr-100_index-{index}"

        align_col = f"ring_density_incr-100_index-{index}"
        start = np.min(table.loc[table[align_col]>=1000]['time_since_begin'])
        if not np.isnan(start):
            table[f'time_since_begin_{index}']=table['time_since_begin']-start
#             ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
#             ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')
            
            x = f'time_since_begin_{index}'
            deb = np.min(table[x])
            end = np.max(table[x])
            time = np.linspace(deb,end,1000)
            tstot +=list(time)
            f = interp1d(table[x], table[column])
            ystot += list(f(time))
            f = interp1d(table[x], table[column2])
            ys2tot+=list(f(time))
            f = interp1d(table[x], table[column3])
            ys3tot+=list(f(time))
df = pd.DataFrame((np.array((tstot,ystot,ys2tot,ys3tot))).transpose(),columns=('ts','ys','ys2','ys3'))
factor = 1
df['ts_round'] = np.floor(df['ts']/factor)*factor
meancurve = df.groupby('ts_round')['ys'].mean()
grad = np.gradient(meancurve,meancurve.index)

ax.plot(meancurve.index,meancurve,linewidth = 5)
meancurve2 = df.groupby('ts_round')['ys2'].mean()
meancurve3 = df.groupby('ts_round')['ys3'].mean()

ax2.plot(meancurve2.index,meancurve2,color='red',label = 'data')
ax2.plot(meancurve2.index,grad/700,color='red',label = r'$\frac{\partial \rho_{data}}{\partial t}$ (scaled)',linestyle="dashed")

# ax2.set_color('red')
ax.set_xlim((-30,30))
ax.set_ylabel(r'$\rho$ ($\mu m.mm^{-2}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel(r'$n$ ($mm^{-2}$)')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
ax2.set_ylim((0,0.15))
plt.legend(fontsize=15)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [114]:
plt.close('all')
fig, ax = plt.subplots()
tstot = []
ystot = []
ys2tot = []
ys3tot = []

ax2 = ax.twinx()
for plate in [94,152,792,799,800,91]:
    table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
    ts = []
    ys = []
    ys2 = []
    for index in range(20):
        column = f"ring_density_incr-100_index-{index}"
#         column = f"ring_active_tips_density_incr-100_index-{index}"
        
        column2 = f"ring_active_tips_density_incr-100_index-{index}"
        column3 = f"mean_speed_incr-100_index-{index}"

        align_col = f"ring_density_incr-100_index-{index}"
        start = np.min(table.loc[table[align_col]>=1000]['time_since_begin'])
        if not np.isnan(start):
            table[f'time_since_begin_{index}']=table['time_since_begin']-start
#             ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
#             ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')
            
            x = f'time_since_begin_{index}'
            deb = np.min(table[x])
            end = np.max(table[x])
            time = np.linspace(deb,end,1000)
            tstot +=list(time)
            f = interp1d(table[x], table[column])
            ystot += list(f(time))
            f = interp1d(table[x], table[column2])
            ys2tot+=list(f(time))
            f = interp1d(table[x], table[column3])
            ys3tot+=list(f(time))
df = pd.DataFrame((np.array((tstot,ystot,ys2tot,ys3tot))).transpose(),columns=('ts','ys','ys2','ys3'))
factor = 1
df['ts_round'] = np.floor(df['ts']/factor)*factor
meancurve = df.groupby('ts_round')['ys'].mean()
grad = np.gradient(meancurve,meancurve.index)

ax.plot(meancurve.index,meancurve,linewidth = 5)
meancurve2 = df.groupby('ts_round')['ys2'].mean()
meancurve3 = df.groupby('ts_round')['ys3'].mean()

ax2.plot(meancurve2.index,meancurve2*meancurve3*2.56,color='red',label = 'data')
ax2.plot(meancurve2.index,grad,color='red',label = r'$\frac{\partial \rho_{data}}{\partial t}$',linestyle="dashed")

# ax2.set_color('red')
ax.set_xlim((-30,30))
ax.set_ylabel(r'$\rho$ ($\mu m.mm^{-2}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel(r'$nv$ ($\mu m.mm^{-2}.h^{-1}$)')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
ax2.set_ylim((0,80))
plt.legend(fontsize=15)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [170]:
plt.close('all')
fig, ax = plt.subplots()
tstot = []
ystot = []
ys2tot = []
ys3tot = []

ax2 = ax.twinx()
for plate in [94,152,792,799,800,91]:
    table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
    ts = []
    ys = []
    ys2 = []
    for index in range(15):
        column = f"ring_anas_density_incr-100_index-{index}"
#         column = f"ring_active_tips_density_incr-100_index-{index}"
        
        column2 = f"ring_active_tips_density_incr-100_index-{index}"
        column3 = f"ring_density_incr-100_index-{index}"
        start = np.min(table.loc[table[column3]>=1000]['time_since_begin'])
        if not np.isnan(start):
            table[f'time_since_begin_{index}']=table['time_since_begin']-start
#             ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
#             ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')
            
            x = f'time_since_begin_{index}'
            deb = np.min(table[x])
            end = np.max(table[x])
            time = np.linspace(deb,end,1000)
            tstot +=list(time)
            f = interp1d(table[x], table[column])
            ystot += list(f(time))
            f = interp1d(table[x], table[column2])
            ys2tot+=list(f(time))
            f = interp1d(table[x], table[column3])
            ys3tot+=list(f(time))
df = pd.DataFrame((np.array((tstot,ystot,ys2tot,ys3tot))).transpose(),columns=('ts','ys','ys2','ys3'))
factor = 1
df['ts_round'] = np.floor(df['ts']/factor)*factor
meancurve = df.groupby('ts_round')['ys'].mean()
ax.plot(meancurve.index,meancurve,linewidth = 5)
meancurve2 = df.groupby('ts_round')['ys2'].mean()
meancurve3 = df.groupby('ts_round')['ys3'].mean()

ax2.plot(meancurve2.index,meancurve2*meancurve3,linewidth = 5,color='red')
# ax2.set_color('red')
ax.set_xlim((-30,30))
ax.set_ylim((0,0.0019))

ax.set_ylabel('anastomosis density ($mm^{-2}.h^{-1}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel(r'$\rho n$ ($\mu m. mm^{-4}$)')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [150]:
np.max(meancurve)/np.max(meancurve2*meancurve3)

2.1900858932606623e-05

In [152]:
# plt.close('all')
# fig, ax = plt.subplots()
tstot = []
ystot = []
ys2tot = []
# ax2 = ax.twinx()
for plate in [94,152,792,799,800,91]:
    table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
    ts = []
    ys = []
    ys2 = []
    for index in range(15):
        column = f"ring_anas_density_incr-100_index-{index}"
        column2 = f"ring_active_tips_density_incr-100_index-{index}"
        start = np.min(table.loc[table[f"ring_density_incr-100_index-{index}"]>=1000]['time_since_begin'])
        if not np.isnan(start):
            table[f'time_since_begin_{index}']=table['time_since_begin']-start
#             ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
#             ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')

            tstot +=table[f'time_since_begin_{index}'].to_list()
            ystot += table[column].astype(float).to_list()
            ys2tot+=table[column2].astype(float).to_list()
            ts +=table[f'time_since_begin_{index}'].to_list()
            ys += table[column].astype(float).to_list()
            ys2+=table[column2].astype(float).to_list()
    df = pd.DataFrame((np.array((ts,ys,ys2))).transpose(),columns=('ts','ys','ys2'))
    factor = 4
    df['ts_round'] = (df['ts']/factor).astype(int)*factor
    meancurve = df.groupby('ts_round')['ys'].mean()
#     ax.plot(meancurve.index,meancurve,color='blue',alpha=0.5)
    meancurve2 = df.groupby('ts_round')['ys2'].mean()
    ax.plot(meancurve.index,meancurve,color='blue',alpha = 0.5)
# df = pd.DataFrame((np.array((tstot,ystot,ys2tot))).transpose(),columns=('ts','ys','ys2'))
# factor = 4
# df['ts_round'] = (df['ts']/factor).astype(int)*factor
# meancurve = df.groupby('ts_round')['ys'].mean()
# ax.plot(meancurve.index,meancurve,linewidth = 5)
# meancurve2 = df.groupby('ts_round')['ys2'].mean()
# ax2.plot(meancurve.index,meancurve2,linewidth = 5,color='red')
# # ax2.set_color('red')
# ax.set_xlim((-30,30))
# ax.set_ylabel('anastomosis density ($mm^{-2}.h{-1}$)')
# ax.set_xlabel('shifted time ($h$)')
# ax2.set_ylabel('active tips density ($mm^{-2}$)')
# ax.tick_params(axis='y', colors='blue')
# ax2.tick_params(axis='y', colors='red')
# plt.legend()

In [172]:
plt.close('all')
fig, ax = plt.subplots()
tstot = []
ystot = []
ys2tot = []
ys3tot = []

ax2 = ax.twinx()
for plate in [94,152,792,799,800,91]:
    table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
    ts = []
    ys = []
    ys2 = []
    for index in range(15):
        column = f"ring_branch_density_incr-100_index-{index}"
#         column = f"ring_active_tips_density_incr-100_index-{index}"
        
        column2 = f"ring_active_tips_density_incr-100_index-{index}"
        column3 = f"ring_density_incr-100_index-{index}"
        start = np.min(table.loc[table[column3]>=1000]['time_since_begin'])
        if not np.isnan(start):
            table[f'time_since_begin_{index}']=table['time_since_begin']-start
#             ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
#             ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')
            
            x = f'time_since_begin_{index}'
            deb = np.min(table[x])
            end = np.max(table[x])
            time = np.linspace(deb,end,1000)
            tstot +=list(time)
            f = interp1d(table[x], table[column])
            ystot += list(f(time))
            f = interp1d(table[x], table[column2])
            ys2tot+=list(f(time))
            f = interp1d(table[x], table[column3])
            ys3tot+=list(f(time))
df = pd.DataFrame((np.array((tstot,ystot,ys2tot,ys3tot))).transpose(),columns=('ts','ys','ys2','ys3'))
factor = 1
df['ts_round'] = np.floor(df['ts']/factor)*factor
meancurve = df.groupby('ts_round')['ys'].mean()
ax.plot(meancurve.index,meancurve,linewidth = 5)
meancurve2 = df.groupby('ts_round')['ys2'].mean()
meancurve3 = df.groupby('ts_round')['ys3'].mean()

ax2.plot(meancurve2.index,meancurve2,linewidth = 5,color='red')
# ax2.set_color('red')
ax.set_xlim((-30,30))
ax.set_ylim((0,0.0074))

ax.set_ylabel('branch density ($mm^{-2}.h^{-1}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel(r'$n$ ($mm^{-2}$)')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [175]:
np.max(meancurve)/np.max(meancurve2)

0.05647099251989463

In [176]:
np.max(meancurve)/np.max(meancurve2)/(2.2e-5)

2566.863296358847

In [168]:
# plt.close('all')
# fig, ax = plt.subplots()
tstot = []
ystot = []
ys2tot = []
# ax2 = ax.twinx()
for plate in [94,152,792,799,800,91]:
    table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
    ts = []
    ys = []
    ys2 = []
    for index in range(15):
        column = f"ring_branch_density_incr-100_index-{index}"
        column2 = f"ring_active_tips_density_incr-100_index-{index}"
        start = np.min(table.loc[table[f"ring_density_incr-100_index-{index}"]>=1000]['time_since_begin'])
        if not np.isnan(start):
            table[f'time_since_begin_{index}']=table['time_since_begin']-start
#             ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
#             ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')

            tstot +=table[f'time_since_begin_{index}'].to_list()
            ystot += table[column].astype(float).to_list()
            ys2tot+=table[column2].astype(float).to_list()
            ts +=table[f'time_since_begin_{index}'].to_list()
            ys += table[column].astype(float).to_list()
            ys2+=table[column2].astype(float).to_list()
    df = pd.DataFrame((np.array((ts,ys,ys2))).transpose(),columns=('ts','ys','ys2'))
    factor = 4
    df['ts_round'] = (df['ts']/factor).astype(int)*factor
    meancurve = df.groupby('ts_round')['ys'].mean()
#     ax.plot(meancurve.index,meancurve,color='blue',alpha=0.5)
    meancurve2 = df.groupby('ts_round')['ys2'].mean()
    ax.plot(meancurve2.index,meancurve,color='blue',alpha = 0.5)
# df = pd.DataFrame((np.array((tstot,ystot,ys2tot))).transpose(),columns=('ts','ys','ys2'))
# factor = 4
# df['ts_round'] = (df['ts']/factor).astype(int)*factor
# meancurve = df.groupby('ts_round')['ys'].mean()
# ax.plot(meancurve.index,meancurve,linewidth = 5)
# meancurve2 = df.groupby('ts_round')['ys2'].mean()
# ax2.plot(meancurve.index,meancurve2,linewidth = 5,color='red')
# # ax2.set_color('red')
# ax.set_xlim((-30,30))
# ax.set_ylabel('anastomosis density ($mm^{-2}.h{-1}$)')
# ax.set_xlabel('shifted time ($h$)')
# ax2.set_ylabel('active tips density ($mm^{-2}$)')
# ax.tick_params(axis='y', colors='blue')
# ax2.tick_params(axis='y', colors='red')
# plt.legend()

In [18]:
plt.close('all')
fig, ax = plt.subplots()
tstot = []
ystot = []
ys2tot = []
ys3tot = []

ax2 = ax.twinx()
for plate in [94,152,792,799,800,91]:
    table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
    ts = []
    ys = []
    ys2 = []
    for index in range(20):
        column = f"ring_anas_density_incr-100_index-{index}"
#         column = f"ring_active_tips_density_incr-100_index-{index}"
        
        column2 = f"ring_active_tips_density_incr-100_index-{index}"
        column3 = f"ring_density_incr-100_index-{index}"
        start = np.min(table.loc[table[column3]>=1000]['time_since_begin'])
        if not np.isnan(start):
            table[f'time_since_begin_{index}']=table['time_since_begin']-start
#             ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
#             ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')
            
            x = f'time_since_begin_{index}'
            deb = np.min(table[x])
            end = np.max(table[x])
            time = np.linspace(deb,end,1000)
            tstot +=list(time)
            f = interp1d(table[x], table[column])
            ystot += list(f(time))
            f = interp1d(table[x], table[column2])
            ys2tot+=list(f(time))
            f = interp1d(table[x], table[column3])
            ys3tot+=list(f(time))
df = pd.DataFrame((np.array((tstot,ystot,ys2tot,ys3tot))).transpose(),columns=('ts','ys','ys2','ys3'))
factor = 1
df['ts_round'] = np.floor(df['ts']/factor)*factor
meancurve = df.groupby('ts_round')['ys'].mean()
ax.plot(meancurve.index,meancurve,linewidth = 5)
meancurve2 = df.groupby('ts_round')['ys2'].mean()
meancurve3 = df.groupby('ts_round')['ys3'].mean()

ax2.plot(meancurve2.index,meancurve2*meancurve3,linewidth = 5,color='red')
# ax2.set_color('red')
ax.set_xlim((-30,30))
ax.set_ylabel('anastomosis density ($mm^{-2}.h{-1}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel(r'$\rho n$ ($\mu m. mm^{-}$)')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …